# 📊 Data Cleaning & Validation - Portfolio Project

**Auteur**: Paul Frette  
**Objectif**: Démonstration de techniques de nettoyage et validation de données  
**Technologies**: Python, Pandas, NumPy, Regex

---

## 🎯 Contexte

Ce notebook démontre mes compétences en nettoyage et validation de données sur un cas d'usage réel :
- Import et nettoyage de données clients provenant de sources multiples
- Standardisation des formats
- Validation des emails et téléphones avec regex
- Analyse de qualité des données

---

## 1️⃣ Import des Librairies

In [ ]:
# Librairies standard
import pandas as pd
import numpy as np
import re

# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

print("✅ Librairies importées avec succès")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 NumPy version: {np.__version__}")

## 2️⃣ Création de Données de Démonstration

Pour ce portfolio, je crée un jeu de données fictif représentatif du type de données que j'ai traité.

In [ ]:
# Création d'un dataset fictif pour démonstration
np.random.seed(42)

data = {
    'ID': range(1, 101),
    'First Name': ['John', 'Jane', 'Bob', 'Alice', 'Charlie'] * 20,
    'Last Name': ['Doe', 'Smith', 'Johnson', 'Williams', 'Brown'] * 20,
    'Email': [
        'john.doe@email.com', 'jane.smith@company.fr', 'bob@invalid',
        'alice.williams@test.com', 'charlie.brown@example.org'
    ] * 20,
    'Phone': [
        '0612345678', '06-12-34-56-78', '0612345', 
        '0612345678', 'invalid_phone'
    ] * 20,
    'Mobile Phone': [None, '0698765432', None, '0698765432', None] * 20,
    'Phone1': ['0612345678', None, None, None, '0687654321'] * 20,
    'Gender': ['Male', 'Female', 'male', 'FEMALE', None] * 20,
    'Created At': pd.date_range('2022-01-01', periods=100, freq='D'),
    'Type Contact': ['individual', 'company', 'Individual', 'Company', None] * 20
}

df = pd.DataFrame(data)

print(f"✅ Dataset créé: {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

## 3️⃣ Nettoyage des Noms de Colonnes

**Problème**: Les colonnes ont des espaces et des majuscules incohérentes.  
**Solution**: Standardisation avec regex.

In [ ]:
def clean_column_name(column_name):
    """
    Nettoie les noms de colonnes:
    - Remplace caractères spéciaux par underscores
    - Convertit en minuscules
    """
    cleaned_name = re.sub(r"[^a-zA-Z0-9]", "_", column_name)
    cleaned_name = cleaned_name.lower()
    return cleaned_name

# Appliquer sur toutes les colonnes
print("📝 Colonnes avant nettoyage:")
print(df.columns.tolist())

new_column_names = {old: clean_column_name(old) for old in df.columns}
df = df.rename(columns=new_column_names)

print("\n✅ Colonnes après nettoyage:")
print(df.columns.tolist())

## 4️⃣ Sélection du Téléphone Principal

**Problème**: Plusieurs colonnes de téléphone, besoin de sélectionner la plus fiable.  
**Solution**: Fonction avec logique de priorité et validation regex.

In [ ]:
def get_most_probable_phone(row):
    """
    Parcourt les colonnes de téléphone par ordre de priorité
    et retourne le premier numéro valide (10 chiffres).
    """
    phone_columns = ['phone', 'mobile_phone', 'phone1']
    
    for column in phone_columns:
        if column in row.index and isinstance(row[column], str):
            # Vérification: au moins 10 chiffres consécutifs
            if re.search(r'\d{10}', row[column]):
                return row[column]
    return None

# Appliquer la fonction
df['phone_principal'] = df.apply(get_most_probable_phone, axis=1)

print("✅ Téléphone principal extrait")
print(f"Taux de complétude: {df['phone_principal'].notna().sum() / len(df) * 100:.1f}%")
df[['phone', 'mobile_phone', 'phone1', 'phone_principal']].head(10)

## 5️⃣ Conversion de Types avec Gestion d'Erreurs

**Objectif**: Convertir les colonnes vers les types appropriés sans plantage.

In [ ]:
# Conversion sécurisée des types
print("📊 Types avant conversion:")
print(df.dtypes)

# Entiers nullables (accepte les NaN)
df['id'] = pd.to_numeric(df['id'], errors='coerce').astype('Int64')

# Dates
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

# Catégories (économie de mémoire)
df['gender'] = df['gender'].astype('category')
df['type_contact'] = df['type_contact'].astype('category')

# Strings
string_columns = ['first_name', 'last_name', 'email', 'phone_principal']
for col in string_columns:
    if col in df.columns:
        df[col] = df[col].astype(str)

print("\n✅ Types après conversion:")
print(df.dtypes)

print(f"\n💾 Réduction mémoire avec catégories: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

## 6️⃣ Validation d'Emails avec Regex

**Objectif**: Identifier les emails valides/invalides pour nettoyage.

In [ ]:
def is_valid_email(email):
    """
    Valide un email avec regex selon le format standard.
    """
    if not isinstance(email, str) or email == 'nan':
        return False
    
    regex = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    return bool(re.match(regex, email))

# Appliquer la validation
df['email_valid'] = df['email'].apply(is_valid_email)

# Statistiques
valid_count = df['email_valid'].sum()
invalid_count = (~df['email_valid']).sum()

print(f"✅ Emails valides: {valid_count} ({valid_count/len(df)*100:.1f}%)")
print(f"❌ Emails invalides: {invalid_count} ({invalid_count/len(df)*100:.1f}%)")

# Exemples d'emails invalides
print("\n📧 Exemples d'emails invalides:")
print(df[~df['email_valid']][['email']].head())

## 7️⃣ Validation de Numéros de Téléphone

**Format attendu**: 10 chiffres (format français)

In [ ]:
def is_valid_phone(phone):
    """
    Valide un numéro de téléphone (10 chiffres).
    """
    if not isinstance(phone, str) or phone == 'None':
        return False
    
    # Extraire uniquement les chiffres
    digits = re.sub(r'\D', '', phone)
    
    # Vérifier 10 chiffres exactement
    return len(digits) == 10

# Validation
df['phone_valid'] = df['phone_principal'].apply(is_valid_phone)

valid_phones = df['phone_valid'].sum()
invalid_phones = (~df['phone_valid']).sum()

print(f"✅ Téléphones valides: {valid_phones} ({valid_phones/len(df)*100:.1f}%)")
print(f"❌ Téléphones invalides: {invalid_phones} ({invalid_phones/len(df)*100:.1f}%)")

# Exemples invalides
print("\n📞 Exemples de téléphones invalides:")
print(df[~df['phone_valid']][['phone_principal']].head())

## 8️⃣ Analyse de Complétude des Données

**Objectif**: Mesurer la qualité du dataset.

In [ ]:
# Calculer le taux de complétude
completeness = df.notna().sum() / len(df)

# Créer un rapport détaillé
quality_report = pd.DataFrame({
    'Colonne': df.columns,
    'Type': df.dtypes,
    'Valeurs Non-Nulles': df.notna().sum(),
    'Valeurs Nulles': df.isnull().sum(),
    'Taux Complétude (%)': (completeness * 100).round(2)
})

quality_report = quality_report.sort_values('Taux Complétude (%)', ascending=False)

print("📊 Rapport de Qualité des Données")
print("=" * 70)
print(quality_report.to_string(index=False))

## 9️⃣ Visualisation de la Qualité

Graphique visuel du taux de complétude par colonne.

In [ ]:
# Graphique de complétude
plt.figure(figsize=(12, 6))

# Préparer les données pour le graphique
plot_data = quality_report[['Colonne', 'Taux Complétude (%)']].head(10)

plt.barh(plot_data['Colonne'], plot_data['Taux Complétude (%)'], 
         color='steelblue', edgecolor='navy')
plt.xlabel('Taux de Complétude (%)', fontsize=12, fontweight='bold')
plt.ylabel('Colonnes', fontsize=12, fontweight='bold')
plt.title('Taux de Complétude par Colonne', fontsize=14, fontweight='bold')
plt.xlim(0, 105)

# Ajouter une ligne de référence à 95%
plt.axvline(x=95, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Seuil 95%')
plt.legend()

plt.tight_layout()
plt.show()

print("✅ Visualisation générée")

## 🔟 Détection de Doublons

Identifier les doublons sur emails et téléphones.

In [ ]:
# Doublons sur email
email_duplicates = df[df['email_valid']]['email'].duplicated(keep=False).sum()
print(f"📧 Doublons d'emails: {email_duplicates} ({email_duplicates/len(df)*100:.1f}%)")

# Doublons sur téléphone
phone_duplicates = df[df['phone_valid']]['phone_principal'].duplicated(keep=False).sum()
print(f"📞 Doublons de téléphones: {phone_duplicates} ({phone_duplicates/len(df)*100:.1f}%)")

# Afficher des exemples de doublons
if email_duplicates > 0:
    print("\n📋 Exemples de doublons d'emails:")
    duplicate_emails = df[df['email'].duplicated(keep=False)].sort_values('email')
    print(duplicate_emails[['id', 'first_name', 'last_name', 'email']].head(10))

## 📊 Résumé Final

Rapport consolidé de l'analyse de qualité.

In [ ]:
print("" + "=" * 70)
print("📊 RÉSUMÉ DE L'ANALYSE DE QUALITÉ DES DONNÉES")
print("=" * 70)
print(f"\n📈 Dataset: {df.shape[0]} lignes × {df.shape[1]} colonnes")
print(f"\n✅ Emails valides: {df['email_valid'].sum()} / {len(df)} ({df['email_valid'].sum()/len(df)*100:.1f}%)")
print(f"✅ Téléphones valides: {df['phone_valid'].sum()} / {len(df)} ({df['phone_valid'].sum()/len(df)*100:.1f}%)")
print(f"\n🔄 Doublons emails: {email_duplicates}")
print(f"🔄 Doublons téléphones: {phone_duplicates}")
print(f"\n💾 Utilisation mémoire: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
print("\n" + "=" * 70)
print("✨ Analyse terminée avec succès !")
print("=" * 70)

---

## 🎯 Compétences Démontrées

✅ **Nettoyage de données**
- Standardisation des noms de colonnes avec regex
- Sélection intelligente de données multiples sources

✅ **Validation de données**
- Regex pour emails (format standard)
- Regex pour téléphones (format français)

✅ **Conversion de types**
- Gestion d'erreurs avec `errors='coerce'`
- Types nullables (`Int64`)
- Optimisation mémoire (catégories)

✅ **Analyse de qualité**
- Taux de complétude
- Détection de doublons
- Visualisations claires

✅ **Bonnes pratiques**
- Code commenté et documenté
- Fonctions réutilisables
- Rapports structurés

---

**Auteur**: Paul Frette  
**Contact**: paul.frette.pro@gmail.com  
**GitHub**: https://github.com/paulfrettepro-collab/data-analysis-portfolio